In [1]:
import pandas as pd
import numpy as np
from statistics import mean
import os
import openpyxl
from openpyxl.workbook import Workbook
import math

In [22]:
#Read in Excel spreadsheet with glass data
df=pd.read_excel('natural_glass_data.xlsx')

#Assign variables for each major element
#Format the major elements as arrays so they can be used in the interpolation
SiO2=(df['SiO2 (wt%)'])
xSi=SiO2.to_numpy()
TiO2=(df['TiO2 (wt%)']) 
xTi=TiO2.to_numpy()
Al2O3=(df['Al2O3 (wt%)'])
xAl=Al2O3.to_numpy()
FeO=(df['FeO(t) (wt%)'])
xFe=FeO.to_numpy()
MgO=(df['MgO (wt%)'])
xMg=MgO.to_numpy()
CaO=(df['CaO (wt%)'])
xCa=CaO.to_numpy()
Na2O=(df['Na2O (wt%)'])
xNa=Na2O.to_numpy()
K2O=(df['K2O (wt%)'])
xK=K2O.to_numpy()
MnO=(df['MnO (wt%)'])
xMn=MnO.to_numpy()

#Load in alphaMELTS liquid composition output, and save it as an Excel file for ease of access
melts=pd.read_table("Liquid_comp_tbl.txt",sep=' ')
melts.to_excel("melts_liquid.xlsx", index=True)
liquid=pd.read_excel("melts_liquid.xlsx",skiprows=1)
name='Test_RhyoliteMELTSmodel'

#Now calculating the residuals, for each major element. The first one is commented to explain the method.
    
#SiO2
six=liquid['MgO']
siy=liquid['SiO2'] #Read in the MgO and SiO2 values. They will be in the order of decreasing T, so following evolution of the liquid. 
b=siy.size #This tells you how many T steps the model ran. Useful to check if it crashed or ran to completion.
sixp=six.to_numpy()
sifp=siy.to_numpy() #Convert the list to numpy to use it in the numpy interpolation package.
si_arr=np.interp(-xMg,-sixp,sifp) #Create array. Inputs are x (=MgO) and y (=SiO2) values from MELTS models, and the MgO contents from the literature. 
#The output is SiO2 values calculated for each MgO value in the literature by interpolating between Melts points
#The xMg and sixp is negative because the x value must be increasing but as MELTS outputs MgO in temp intervals it will decrease as temp decreases. 
sires=math.sqrt(np.nansum(np.square(xSi - si_arr))/len(SiO2)) #The residual. Formula is the Root mean square error. Calculated by subtracting each SiO2 content in literature from the SiO2 content predicted by Melts, for each MgO value in the literature
nsires=sires/mean(xSi) #Normalise the oxide RMSE to the average value of the oxide in the natural data
    
#Repeat for other major elements 
#TiO2
tix=liquid['MgO']
tiy=liquid['TiO2']
tixp=tix.to_numpy()
tifp=tiy.to_numpy()
ti_arr=np.interp(-xMg,-tixp,tifp)
tires=math.sqrt(np.nansum(np.square(TiO2 - ti_arr))/len(TiO2))
ntires=tires/mean(xTi)
    
#FeOt
fex=liquid['MgO']
fey=(liquid['FeO']+(liquid['Fe2O3']*0.8998)) #Fe content is reported as FeO(T) in the literature but as FeO and Fe2O3 in Melts outputs, so use the conversion to get total Fe. 
fexp=fex.to_numpy()
fefp=fey.to_numpy()
fe_arr=np.interp(-xMg,-fexp,fefp)
feres=math.sqrt(np.nansum(np.square(FeO - fe_arr))/len(FeO))
nferes=feres/mean(xFe)
    
#CaO
cax=liquid['MgO']
cay=(liquid['CaO'])
caxp=cax.to_numpy()
cafp=cay.to_numpy()
ca_arr=np.interp(-xMg,-caxp,cafp)
cares=math.sqrt(np.nansum(np.square(CaO - ca_arr))/len(CaO))
ncares=cares/mean(xCa)
   
#Na2O
nax=liquid['MgO']
nay=(liquid['Na2O'])
naxp=nax.to_numpy()
nafp=nay.to_numpy()
na_arr=np.interp(-xMg,-naxp,nafp)
nares=math.sqrt(np.nansum(np.square(Na2O - na_arr))/len(Na2O))
nnares=nares/mean(xNa)
    
#Al2O3
alx=liquid['MgO']
aly=(liquid['Al2O3'])
alxp=alx.to_numpy()
alfp=aly.to_numpy()
al_arr=np.interp(-xMg,-alxp,alfp)
alres=math.sqrt(np.nansum(np.square(Al2O3 - al_arr))/len(Al2O3))
nalres=alres/mean(xAl)
    
#K2O
kx=liquid['MgO']
ky=(liquid['K2O'])
kxp=kx.to_numpy()
kfp=ky.to_numpy()
k_arr=np.interp(-xMg,-kxp,kfp)
kres=math.sqrt(np.nansum(np.square(K2O - k_arr))/len(K2O))
nkres=kres/mean(xK)
  
#MnO
mnx=liquid['MgO']
mny=(liquid['MnO'])
mnxp=mnx.to_numpy()
mnfp=mny.to_numpy()
mn_arr=np.interp(-xMg,-mnxp,mnfp)
mnres=math.sqrt(np.nansum(np.square(MnO - mn_arr))/len(MnO))
nmnres=mnres/mean(xMn)

#Calculate total RMSE
rmse = nsires+ntires+nferes+ncares+nnares+nalres+nkres+nmnres
print(rmse)


#Save the errors for each major element, and the RMSE, into a spreadsheet
myworkbook=openpyxl.load_workbook('CalculatedRMSE.xlsx')
worksheet= myworkbook['Sheet1']
worksheet.append([nsires,ntires,nferes,ncares,nnares,nalres,nkres,nmnres,name,b,rmse])
myworkbook.save('CalculatedRMSE.xlsx')




2.4007879930262837
